## Battle of Neighborhoods

### Analysis of Montreal

First start by scraping data

In [13]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values


# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library



In [14]:
url = 'https://en.wikipedia.org/wiki/Boroughs_of_Montreal'
url1 = 'https://news.shupilov.com/blog/montreal-boroughs-ranked-according-to-5-year-appreciation/'
page = requests.get(url).text
page1 = requests.get(url1).text
soup = BeautifulSoup(page, 'lxml')
soup1 = BeautifulSoup(page1, 'lxml')


In [15]:
table = soup.find('table', class_='sortable wikitable ')

A=[]
B=[]
C=[]
D=[]

for row in soup.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==5:
        A.append(cells[1].find(text=True))
        B.append(cells[2].find(text=True))
        C.append(cells[3].find(text=True))
        D.append(cells[4].find(text=True))
    
        
df=pd.DataFrame(A,columns=['Borough'])
df['Population']=B
df['Area']=C
df['Density']=D

df



,Borough,Population,Area,Density
0,Ahuntsic-Cartierville,"134,245",24.2,"5,547.3"
1,Anjou,"42,796",13.7,"3,123.8"
2,Côte-des-Neiges–Notre-Dame-de-Grâce,"166,520",21.4,"7,781.3"
3,Lachine,"44,489",17.7,"2,513.5"
4,LaSalle,"76,853",16.3,"4,714.9"
5,Le Plateau-Mont-Royal,"104,000",8.1,"12,839.5"
6,Le Sud-Ouest,"78,151",15.7,"4,977.8"
7,L'Île-Bizard–Sainte-Geneviève,"18,413",23.6,780.2
8,Mercier–Hochelaga-Maisonneuve,"136,024",25.4,"5,355.3"
9,Montréal-Nord,"84,234",11.1,"7,588.6"


Now scrape the second table

In [17]:
table1 = soup1.find('table', class_='sortable wikitable ')

A1=[]
B1=[]


for row1 in soup1.findAll('tr'):
    cells1 =row1.findAll('td')
    if len(cells1)==4:
        A1.append(cells1[1].find(text=True))
        B1.append(cells1[2].find(text=True))
        
        
df1 = pd.DataFrame(A1,columns=['Borough'])
df1['Average Price']=B1
df1.drop(axis = 0, index = 0, inplace = True)
df1

,Borough,Average Price
1,Rosemont/La Petite-Patrie,"$585,431"
2,Verdun,"$536,734"
3,Beaconsfield,"$704,917"
4,Pointe-Claire,"$479,476"
5,Le Sud-Ouest,"$525,435"
6,Kirkland,"$607,191"
7,Lachine,"$442,126"
8,Île-des-Soeurs,"$961,576"
9,Mont-Royal,"$1,281,453"
10,Dollard-Des Ormeaux,"$488,793"


I used Google map to find the latitude and longitude of each borough in Montreal

In [27]:
location = pd.read_csv(r'C:\Users\bianb\Documents/Montreal_location.csv')
location

,Borough,Latitude,Longitude
0,Ahuntsic-Cartierville,45.538132,-73.668279
1,Anjou,45.615474,-73.575762
2,Côte-des-Neiges–Notre-Dame-de-Grâce,45.493032,-73.633404
3,Lachine,45.445746,-73.685329
4,LaSalle,45.434177,-73.613079
5,Le Plateau-Mont-Royal,45.527734,-73.588190
6,Le Sud-Ouest,45.459012,-73.591131
7,L'Île-Bizard–Sainte-Geneviève,45.486650,-73.899608
8,Mercier–Hochelaga-Maisonneuve,45.585585,-73.527262
9,Montréal-Nord,45.599943,-73.624871


Next we are going to join the tables together with the location data of each borough

In [ ]:
# #first import the geocoder
# #!pip install geocoder
# import geocoder 
# # initialize your variable to None
# lat_lng_coords = None

# # loop until you get the coordinates
# while(lat_lng_coords is None):
#   g = geocoder.google('{}, Montreal, Quebec'.format(Borough))
#   lat_lng_coords = g.latlng

# latitude = lat_lng_coords[0]
# longitude = lat_lng_coords[1]

In [29]:
df_mont = pd.merge(df, df1, on = 'Borough', how = 'inner')
df_montreal = pd.merge(df_mont, location, on = 'Borough', how = 'inner')
df_montreal

,Borough,Population,Area,Density,Average Price,Latitude,Longitude
0,Ahuntsic-Cartierville,"134,245",24.2,"5,547.3","$565,560",45.538132,-73.668279
1,Anjou,"42,796",13.7,"3,123.8","$396,957",45.615474,-73.575762
2,Lachine,"44,489",17.7,"2,513.5","$442,126",45.445746,-73.685329
3,LaSalle,"76,853",16.3,"4,714.9","$447,021",45.434177,-73.613079
4,Le Plateau-Mont-Royal,"104,000",8.1,"12,839.5","$838,636",45.527734,-73.588190
5,Le Sud-Ouest,"78,151",15.7,"4,977.8","$525,435",45.459012,-73.591131
6,Montréal-Nord,"84,234",11.1,"7,588.6","$299,472",45.599943,-73.624871
7,Pierrefonds-Roxboro,"69,297",27.1,"2,557.1","$376,115",45.459955,-73.882112
8,Saint-Laurent,"98,828",42.8,"2,309.1","$606,828",45.504889,-73.713050
9,Verdun,"69,229",9.7,"7,137.0","$536,734",45.459382,-73.570233


In [42]:
df_montreal.to_csv('df_montreal.csv')

Next I am going to use the FourSquare API 

In [19]:
CLIENT_ID = 'BYDVXOLP1M0EQHCNSJEXRGO3X0UGYT4NVRCINABDZMHR5G5Y' # your Foursquare ID
CLIENT_SECRET = 'V4PSJURPUSEDUHGYRGIKWTBAVQRHW1DPYIFQVGSICQVJ4MVM' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BYDVXOLP1M0EQHCNSJEXRGO3X0UGYT4NVRCINABDZMHR5G5Y
CLIENT_SECRET:V4PSJURPUSEDUHGYRGIKWTBAVQRHW1DPYIFQVGSICQVJ4MVM


In [20]:
# find the location for Montreal
address = 'Montreal, QC'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Montreal are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Montreal are 45.4972159, -73.6103642.


In [43]:


# create map of Montreal using latitude and longitude values
map_montreal = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng,label in zip(df_montreal['Latitude'], df_montreal['Longitude'], df_montreal['Average Price']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_montreal)  
    
map_montreal